In [1]:
# https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
# https://medium.com/datadriveninvestor/choosing-the-best-algorithm-for-your-classification-model-7c632c78f38f
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
# https://www.dataquest.io/blog/sci-kit-learn-tutorial/
# https://datascience.stackexchange.com/questions/33256/how-to-apply-machine-learning-model-to-new-dataset
# https://speakerdeck.com/datasciencela/tianqi-chen-xgboost-overview-and-latest-news-la-meetup-talk?slide=33
# https://github.com/ritvikkhanna09/Census-classifier-comparison/blob/master/NeuralNetworks.ipynb
# https://towardsdatascience.com/k-as-in-keras-simple-classification-model-a9d2d23d5b5a

In [2]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler


from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from xgboost import plot_importance
from xgboost import plot_tree
import xgboost as xgb

import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2

Using TensorFlow backend.


In [3]:
path = Path('/Users/bbabu/fastai/DataSamples')

In [4]:
# data = pd.read_csv(path/'Groups_with_Owners-V1.csv', sep=';', header=0)
# data['Owner'].replace(['No','Yes'],[0,1],inplace=True)
# data = data[data.groupby('GroupDN')['Owner'].transform(lambda x: x.sum() >= 1)]
# data['Owner'].replace([0,1],['No','Yes'],inplace=True)
# data.to_csv(path/'Groups_with_Owners-V1a.csv', header=True, index=False, sep=';')

In [5]:
# df_test = pd.read_csv(path/'Test-Group-2.csv', sep=',', header=0)
# df_test_2 = pd.read_csv(path/'Test-Group-2.csv', sep=',', header=0)

In [6]:
# data = pd.read_csv(path/'Groups_with_Owners-V2.csv', sep=',', header=0)
# df_test = pd.read_csv(path/'Groups_without_Owners-V2.csv', sep=',', header=0)
# df_test_2 = pd.read_csv(path/'Groups_without_Owners-V2.csv', sep=',', header=0)

In [7]:
data = pd.read_csv(path/'Groups_with_Owners-V1a.csv', sep=';', header=0)
df_test = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)
df_test_2 = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)

In [8]:
data.drop(['Description', 'MemberDN', 'State'], axis=1, inplace=True)
df_test.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)
df_test_2.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)

In [9]:
# data.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)
# df_test.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)
# df_test_2.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)

In [10]:
# data.drop(['GroupCountry'], axis=1, inplace=True)
# df_test.drop(['GroupCountry'], axis=1, inplace=True)
# df_test_2.drop(['GroupCountry'], axis=1, inplace=True)

In [11]:
data.drop_duplicates(keep='first',inplace=True) 
df_test.drop_duplicates(keep='first',inplace=True)
df_test_2.drop_duplicates(keep='first',inplace=True) 

In [12]:
data.size

6645350

In [13]:
# print(data.dtypes)
# print(df_test.dtypes)
# print(df_test_2.dtypes)

In [14]:
le = preprocessing.LabelEncoder()
categorical = list(data.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    data[cat].fillna('NaN', inplace=True)
    data[cat] = le.fit_transform(data[cat].astype(str))


In [15]:
le = preprocessing.LabelEncoder()

categorical = list(df_test_2.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    df_test_2[cat].fillna('NaN', inplace=True)
    df_test_2[cat] = le.fit_transform(df_test_2[cat].astype(str))


In [16]:
# data.set_index('GroupDN', drop=False, inplace=True)

In [17]:
#Create Dependent and Independent Datasets based on our Dependent #and Independent features
# X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City', 'GroupCountry', 'GroupDomain', 'Domain']]
# X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City', 'GroupDomain', 'Domain']].values
X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City']]
y = data['Owner'].values
#Split the Data into Training and Testing sets with test size as #33%
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, shuffle=False)
eval_set = [(X_train, y_train), (X_test, y_test)]

In [18]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
# X_train.values

In [20]:
# data.reindex(columns=)

In [21]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 100, init = 'uniform', activation = 'relu', input_dim = X_test.shape[1]))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 100, init = 'uniform', activation = 'relu'))
classifier.add(Dense(output_dim = 100, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
                   
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100, verbose = 1)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/Users/bbabu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=9, units=100, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/bbabu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  
/Users/bbabu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  import sys
/Users/bbabu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  if __name__ == '__main__':


Epoch 1/100
445238/445238 [==============================] - 529s 1ms/step - loss: 0.1622 - accuracy: 0.9585
Epoch 2/100
445238/445238 [==============================] - 526s 1ms/step - loss: 0.1579 - accuracy: 0.9585
Epoch 3/100
445238/445238 [==============================] - 547s 1ms/step - loss: 0.1558 - accuracy: 0.9586
Epoch 4/100
445238/445238 [==============================] - 562s 1ms/step - loss: 0.1544 - accuracy: 0.9587
Epoch 5/100
445238/445238 [==============================] - 562s 1ms/step - loss: 0.1533 - accuracy: 0.9586
Epoch 6/100
445238/445238 [==============================] - 565s 1ms/step - loss: 0.1522 - accuracy: 0.9587
Epoch 7/100
445238/445238 [==============================] - 568s 1ms/step - loss: 0.1517 - accuracy: 0.9587
Epoch 8/100
445238/445238 [==============================] - 563s 1ms/step - loss: 0.1509 - accuracy: 0.9587
Epoch 9/100
445238/445238 [==============================] - 567s 1ms/step - loss: 0.1502 - accuracy: 0.9588
Epoch 10/100
445238

445238/445238 [==============================] - 131s 294us/step - loss: 0.1597 - accuracy: 0.9586
Epoch 75/100
445238/445238 [==============================] - 131s 294us/step - loss: 0.1590 - accuracy: 0.9586
Epoch 76/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1617 - accuracy: 0.9586
Epoch 77/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1654 - accuracy: 0.9586
Epoch 78/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1586 - accuracy: 0.9586
Epoch 79/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1653 - accuracy: 0.9586
Epoch 80/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1611 - accuracy: 0.9586
Epoch 81/100
445238/445238 [==============================] - 131s 295us/step - loss: 0.1617 - accuracy: 0.9586
Epoch 82/100
445238/445238 [==============================] - 131s 294us/step - loss: 0.1637 - accuracy: 0.9585
Epoch

In [22]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.02)


# print (classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)

# acc = accuracy_score(y_test, y_pred)
# logloss = log_loss(y_test, y_pred)

In [23]:
loss, acc = classifier.evaluate(X_test, y_test, batch_size = 32)

219297/219297 [==============================] - 6s 28us/step


In [24]:
# Save model to disk
filename = path/'Final_Model-NN-V6.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [25]:
# filename = path/'Final_Model-XGBooster-Saved.sav'

In [26]:
# Load model from disk and use it to make new predictions
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)

In [27]:
X_test = df_test_2.values
loaded_model = pickle.load(open(filename, 'rb'))
X_test = sc.transform(X_test)
pred = loaded_model.predict(X_test)

In [28]:
# pred = (pred > 0.02).astype(int)

In [29]:
final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Owner': pred[:, 0]})

In [30]:
final_df.to_csv(path/'MLProcessed1-NN-V6.csv', header=True, index=False)

In [31]:
grouped = final_df.groupby(['GroupDN'])

In [32]:
idx = final_df.groupby(['GroupDN'])['Owner'].transform(max) == final_df['Owner']
final_df[idx].to_csv(path/'MLProcessed1-NN-Grouped.csv', header=True, index=False)

In [33]:
i = 0
for name, group in grouped:
    if group['Owner'].sum() == 0:
        i += 1
        

In [34]:
print("Total number of predicted owners are: %s"%(len(pred)))
print("Ratio of owners are: %s"%(len(pred)/len(grouped)))
print("Mean of owners are: %s"%(pred.mean()))
print("Total number of groups are: %s"%(len(grouped)))
print("Number of predicted groups with owners are: %s"%(len(grouped) - i))
print("")
print("Model accuracy is: %s"%(acc))
print('Model loss is:', loss)
print("")
print("Confusion matrix of the model is:")
print(cm)
print("")
print("Classification report for the model is:")
print (classification_report(y_test, y_pred))
print("")
print("Summary of the model is:")
classifier.summary()

Total number of predicted owners are: 909279
Ratio of owners are: 18.570737087188284
Mean of owners are: 0.021149436
Total number of groups are: 48963
Number of predicted groups with owners are: 37799

Model accuracy is: 0.9633054733276367
Model loss is: 0.1941447409156126

Confusion matrix of the model is:
[[ 88835 122415]
 [  2393   5654]]

Classification report for the model is:
              precision    recall  f1-score   support

           0       0.97      0.42      0.59    211250
           1       0.04      0.70      0.08      8047

    accuracy                           0.43    219297
   macro avg       0.51      0.56      0.34    219297
weighted avg       0.94      0.43      0.57    219297


Summary of the model is:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1000      
__________________________________________

In [35]:
# df_test_2.to_csv(path/'Encoded_without_Owners.csv', header=True, index=False)
# data.to_csv(path/'Encoded_with_Owners.csv', header=True, index=False)